<a href="https://colab.research.google.com/github/vkuma13/Vicky_Port1/blob/main/AMEX_Credit_Card_Default_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
!pip install imblearn
from imblearn.over_sampling import SMOTE
from random import shuffle
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn.ensemble import RandomForestClassifier

import zipfile

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
data = pd.read_csv('/content/train.csv')

data






FileNotFoundError: ignored

In [ ]:
data1 = pd.read_csv('/content/train.csv')
data1

In [ ]:
#Data Enginnering 

data.isna().sum()

In [ ]:
data.no_of_children.hist()
data.no_of_children.fillna(np.median(data.no_of_children.dropna()),inplace= True)

In [ ]:
data.no_of_days_employed.hist()

data.no_of_days_employed.fillna(np.median(data.no_of_days_employed.dropna()), inplace=True)

In [ ]:
data.total_family_members.hist()
data.total_family_members.fillna(np.median(data.total_family_members.dropna()), inplace=True)

In [ ]:
data.yearly_debt_payments.hist()
data.yearly_debt_payments.fillna(np.median(data.yearly_debt_payments.dropna()), inplace=True)

In [ ]:
data.credit_score.hist()
data.credit_score.fillna(np.median(data.credit_score.dropna()), inplace=True)

In [ ]:
data.isna().sum()  

In [ ]:
data.owns_car.fillna(data.owns_car.mode().values[0],inplace=True)
data.migrant_worker.fillna(data.migrant_worker.mode().values[0],inplace=True)

In [ ]:
data

In [ ]:
#Synthetic Feartures: Generating meaningful synthetic features from the data

def get_profit_category(value):
    if value < 0 :
        return "Loss"
    elif value == 0 : 
        return "Zero"
    else:
        return "Profit"

data["in_hand_balance"] = (data.net_yearly_income - data.yearly_debt_payments)
data["in_profit"] = [get_profit_category(i) for i in data["in_hand_balance"].values]
data["total_income_received"] = data.net_yearly_income * data.no_of_days_employed
data["employment_years"] = (data.no_of_days_employed / 365.0)

In [ ]:
data.employment_years.clip(upper = np.median(data.employment_years), inplace=True)
# numpy.clip() function is used to Clip (limit) the values in an array.

In [ ]:
#Exploratory Data Analysis

cnames = {
'antiquewhite':         '#FAEBD7',
'aqua':                 '#00FFFF',
'aquamarine':           '#7FFFD4',
'beige':                '#F5F5DC',
'bisque':               '#FFE4C4',
'black':                '#000000',
'blanchedalmond':       '#FFEBCD',
'blue':                 '#0000FF',
'blueviolet':           '#8A2BE2',
'brown':                '#A52A2A',
'burlywood':            '#DEB887',
'cadetblue':            '#5F9EA0',
'chartreuse':           '#7FFF00',
'chocolate':            '#D2691E',
'coral':                '#FF7F50',
'cornflowerblue':       '#6495ED',
'crimson':              '#DC143C',
'cyan':                 '#00FFFF',
'darkblue':             '#00008B',
'darkcyan':             '#008B8B',
'darkgoldenrod':        '#B8860B',
'darkgray':             '#A9A9A9',
'darkgreen':            '#006400',
'darkkhaki':            '#BDB76B',
'darkmagenta':          '#8B008B',
'darkolivegreen':       '#556B2F',
'darkorange':           '#FF8C00',
'darkorchid':           '#9932CC',
'darkred':              '#8B0000',
'darksalmon':           '#E9967A',
'darkseagreen':         '#8FBC8F',
'darkslateblue':        '#483D8B',
'darkslategray':        '#2F4F4F',
'darkturquoise':        '#00CED1',
'darkviolet':           '#9400D3',
'deeppink':             '#FF1493',
'deepskyblue':          '#00BFFF',
'dimgray':              '#696969',
'dodgerblue':           '#1E90FF',
'firebrick':            '#B22222',
'forestgreen':          '#228B22',
'fuchsia':              '#FF00FF',
'gainsboro':            '#DCDCDC',
'gold':                 '#FFD700',
'goldenrod':            '#DAA520',
'gray':                 '#808080',
'green':                '#008000',
'greenyellow':          '#ADFF2F',
'honeydew':             '#F0FFF0',
'hotpink':              '#FF69B4',
'indianred':            '#CD5C5C',
'indigo':               '#4B0082',
'ivory':                '#FFFFF0',
'khaki':                '#F0E68C',
'lavender':             '#E6E6FA',
'lawngreen':            '#7CFC00',
'lemonchiffon':         '#FFFACD',
'lightblue':            '#ADD8E6',
'lightcoral':           '#F08080',
'lightcyan':            '#E0FFFF',
'lightgoldenrodyellow': '#FAFAD2',
'lightgreen':           '#90EE90',
'lightgray':            '#D3D3D3',
'lightpink':            '#FFB6C1',
'lightsalmon':          '#FFA07A',
'lightseagreen':        '#20B2AA',
'lightskyblue':         '#87CEFA',
'lightslategray':       '#778899',
'lightsteelblue':       '#B0C4DE',
'lime':                 '#00FF00',
'limegreen':            '#32CD32',
'magenta':              '#FF00FF',
'maroon':               '#800000',
'mediumaquamarine':     '#66CDAA',
'mediumblue':           '#0000CD',
'mediumorchid':         '#BA55D3',
'mediumpurple':         '#9370DB',
'mediumseagreen':       '#3CB371',
'mediumslateblue':      '#7B68EE',
'mediumspringgreen':    '#00FA9A',
'mediumturquoise':      '#48D1CC',
'mediumvioletred':      '#C71585',
'midnightblue':         '#191970',
'mintcream':            '#F5FFFA',
'moccasin':             '#FFE4B5',
'navajowhite':          '#FFDEAD',
'navy':                 '#000080',
'olive':                '#808000',
'olivedrab':            '#6B8E23',
'orange':               '#FFA500',
'orangered':            '#FF4500',
'orchid':               '#DA70D6',
'palegoldenrod':        '#EEE8AA',
'palegreen':            '#98FB98',
'paleturquoise':        '#AFEEEE',
'palevioletred':        '#DB7093',
'papayawhip':           '#FFEFD5',
'peachpuff':            '#FFDAB9',
'peru':                 '#CD853F',
'pink':                 '#FFC0CB',
'plum':                 '#DDA0DD',
'powderblue':           '#B0E0E6',
'purple':               '#800080',
'red':                  '#FF0000',
'rosybrown':            '#BC8F8F',
'royalblue':            '#4169E1',
'saddlebrown':          '#8B4513',
'salmon':               '#FA8072',
'sandybrown':           '#FAA460',
'seagreen':             '#2E8B57',
'sienna':               '#A0522D',
'silver':               '#C0C0C0',
'skyblue':              '#87CEEB',
'slateblue':            '#6A5ACD',
'slategray':            '#708090',
'springgreen':          '#00FF7F',
'steelblue':            '#4682B4',
'tan':                  '#D2B48C',
'teal':                 '#008080',
'thistle':              '#D8BFD8',
'tomato':               '#FF6347',
'turquoise':            '#40E0D0',
'violet':               '#EE82EE',
'wheat':                '#F5DEB3',
'white':                '#FFFFFF',
'yellow':               '#FFFF00',
'yellowgreen':          '#9ACD32'}

colors = list(cnames.keys())

In [ ]:
data.columns

In [ ]:
plt.figure(figsize=(10,8))
shuffle(colors)
plt.subplot(2,2,1)
plt.hist(data.age,color=colors[0])
plt.xlabel("Age group")
plt.ylabel("Count")
plt.title("Which age group is dominant?")

shuffle(colors)
plt.subplot(2,2,2)
plt.hist(data[data.credit_card_default == 0 ].age,color=colors[0])
plt.xlabel("Age group")
plt.ylabel("Count")
plt.title("Which age group is dominant among the non-defaulters?")

shuffle(colors)
plt.subplot(2,2,3)
plt.hist(data[data.credit_card_default == 1].age,color=colors[0])
plt.xlabel("Age group")
plt.ylabel("Count")
plt.title("Which age group is dominant among the defaulters?")
plt.tight_layout()


**Among** the above case the 25,40,55 is Dominant ages

---

Which Gender is Dominant ?



In [ ]:
plt.figure(figsize= (10,9)) 
plt.subplot(2,2,1)
shuffle(colors)
un , count = np.unique(data.gender , return_counts = True)
plt.bar(un, count, color=colors)
plt.xlabel("Gender")
plt.ylabel("count")
plt.title("which gender group is Dominant ?")

plt.subplot(2,2,2)
shuffle(colors)
un , count = np.unique(data[data.credit_card_default == 1].gender, return_counts = True)
plt.bar(un, count,color = colors)
plt.xlabel("Gender")
plt.ylabel("count")
plt.title("which gender group is Dominant among defaulters ?")

plt.subplot(2,2,3)
shuffle(colors)
un, count = np.unique(data[data.credit_card_default == 0].gender, return_counts=True)
plt.bar(un, count, color=colors)
plt.xlabel("Gender")
plt.ylabel("Count")
plt.title("Which gender group is dominant among the non-defaulters?")

plt.tight_layout()






 Females tend to be more Non-defaulters while Males tend to be more in number in Defaulters.


---

How many owns a vehicle?

In [ ]:
plt.figure(figsize=(10,9))
plt.subplot(2,2,1)
shuffle(colors)
un , count = np.unique(data.owns_car.dropna(),return_counts=True)
un = ["Yes" if i.lower() == 'y' else "no" for i in un]
plt.bar(un,count,color = colors)
plt.xlabel("Vehicle Ownership")
plt.ylabel("count")
plt.title("how own a cars")

plt.subplot(2,2,2)
shuffle(colors)
un, count = np.unique(data[data.credit_card_default == 1].owns_car.dropna(), return_counts=True)
un = ["Yes" if i.lower() == "y" else "No" for i in un]
plt.bar(un, count, color=colors)
plt.xlabel("Vehicle Ownership")
plt.ylabel("Count")
plt.title("How many own a car among the defaulters?")

plt.subplot(2,2,3)
shuffle(colors)
un, count = np.unique(data[data.credit_card_default == 0].owns_car.dropna(), return_counts=True)
un = ["Yes" if i.lower() == "y" else "No" for i in un]
plt.bar(un, count, color=colors)
plt.xlabel("Vehicle Ownership")
plt.ylabel("Count")
plt.title("How many own a car among the non-defaulters?")
plt.tight_layout()


Among both classes, majority do not own a car.


---
How many have a residence ownership?


In [ ]:
plt.figure(figsize=(10,9))
plt.subplot(2,2,1)
shuffle(colors)
un, count = np.unique(data.owns_house.dropna(), return_counts=True)
un = ["Yes" if i.lower() == "y" else "No" for i in un]
plt.bar(un, count, color=colors)
plt.xlabel("House Ownership")
plt.ylabel("Count")
plt.title("How many owns a residence?")

plt.subplot(2,2,2)
shuffle(colors)
un, count = np.unique(data[data.credit_card_default == 1].owns_house.dropna(), return_counts=True)
un = ["Yes" if i.lower() == "y" else "No" for i in un]
plt.bar(un, count, color=colors)
plt.xlabel("House Ownership")
plt.ylabel("Count")
plt.title("How many owns a residence among the defaulters?")

plt.subplot(2,2,3)
shuffle(colors)
un, count = np.unique(data[data.credit_card_default == 0].owns_house.dropna(), return_counts=True)
un = ["Yes" if i.lower() == "y" else "No" for i in un]
plt.bar(un, count, color=colors)
plt.xlabel("House Ownership")
plt.ylabel("Count")
plt.title("How many owns a residence among the non-defaulters?")
plt.tight_layout()

Majority Own a residence among the defaulters while the opposite happens in the case of Non-defaulters


---
How many children do they have?


In [ ]:
plt.figure(figsize=(10,9))
plt.subplot(2,2,1)
plt.hist(data.no_of_children,color='brown')
plt.xlabel("No of Children")
plt.ylabel("Count")
plt.title("How many children do they have?")
plt.subplot(2,2,2)
plt.hist(data[data.credit_card_default == 1].no_of_children,color='brown')
plt.xlabel("No of Children")
plt.ylabel("Count")
plt.title("How many children do defaulters have?")
plt.subplot(2,2,3)
plt.hist(data[data.credit_card_default == 0].no_of_children,color='brown')
plt.xlabel("No of Children")
plt.ylabel("Count")
plt.title("How many children do defaulters have?")
plt.tight_layout()

The distribution stays the same among both classes with negligible variations.

---

How much do they earn?

In [ ]:
plt.figure(figsize=(10,9))
plt.subplot(2,2,1)
plt.hist(data.net_yearly_income,color='aqua')
plt.xlabel("Income group")
plt.ylabel("Count")
plt.title("How much do they earn?")
plt.subplot(2,2,2)
plt.hist(data[data.credit_card_default == 1].net_yearly_income,color='aqua')
plt.xlabel("Income group")
plt.ylabel("Count")
plt.title("How much do defaulters earn?")
plt.subplot(2,2,3)
plt.hist(data[data.credit_card_default == 0].net_yearly_income,color='aqua')
plt.xlabel("Income group")
plt.ylabel("Count")
plt.title("How much do non-defaulters earn?")
plt.tight_layout()

Counter Intuitively, Defaulters earn in higher amount in comparison with Non-defaulters

---

How long is their work experience?

In [ ]:
plt.figure(figsize=(10,9))
plt.subplot(2,2,1)
plt.hist(data.employment_years,color='blue')
plt.xlabel("Time Span")
plt.ylabel("Count")
plt.title("How long have they been working?")
plt.subplot(2,2,2)
plt.hist(data[data1.credit_card_default == 1].employment_years,color='blue')
plt.xlabel("Time Span")
plt.ylabel("Count")
plt.title("How long have the defaulters been working?")
plt.subplot(2,2,3)
plt.hist(data[data1.credit_card_default == 0].employment_years,color='blue')
plt.xlabel("Time Span")
plt.ylabel("Count")
plt.title("How long have the non-defaulters been working?")
plt.tight_layout()

Somewhat similar distribution with Moderate Work Experience higher among defaulters.

---

What is their work?

In [ ]:
shuffle(colors)
plt.figure(figsize=(15,12))
plt.subplot(2,2,1)
un, count = np.unique(data.occupation_type.dropna(), return_counts=True)
plt.barh(un, count, color=colors)
plt.xlabel("Occupation Type")
plt.ylabel("Count")
plt.title("What profession do they do?")

plt.subplot(2,2,2)
un, count = np.unique(data[data.credit_card_default == 1].occupation_type.dropna(), return_counts=True)
plt.barh(un, count, color=colors)
plt.xlabel("Occupation Type")
plt.ylabel("Count")
plt.title("What profession do defaulters do?")

plt.subplot(2,2,3)
un, count = np.unique(data[data.credit_card_default == 0].occupation_type.dropna(), return_counts=True)
plt.barh(un, count, color=colors)
plt.xlabel("Occupation Type")
plt.ylabel("Count")
plt.title("What profession do non-defaulters do?")
plt.tight_layout()

Comparatively, Defaulters have Low-income jobs compared to Non-Defaulters

---

How big is their family

In [ ]:
plt.figure(figsize=(10,9))
plt.subplot(2,2,1)
plt.hist(data.total_family_members,color='seagreen')
plt.xlabel("Family Member Count")
plt.ylabel("Count")
plt.title("How big family do they have?")
plt.subplot(2,2,2)
plt.hist(data[data.credit_card_default == 1].total_family_members,color='seagreen')
plt.xlabel("Family Member Count")
plt.ylabel("Count")
plt.title("How big family do defaulters have?")
plt.subplot(2,2,3)
plt.hist(data[data.credit_card_default == 0].total_family_members,color='seagreen')
plt.xlabel("Family Member Count")
plt.ylabel("Count")
plt.title("How big family do non-defaulters have?")
plt.tight_layout()

Similar distributions with neglible variations.

---

Are they a migrant?

In [ ]:
shuffle(colors)
plt.figure(figsize=(10,9))
plt.subplot(2,2,1)
un, count = np.unique(data.migrant_worker.dropna(), return_counts=True)
un = [str(int(i)) for i in un]
plt.bar(un, count, color = colors)
plt.xlabel("Migration Status")
plt.ylabel("Count")
plt.title("Are they an outsider?")
plt.subplot(2,2,2)
un, count = np.unique(data[data.credit_card_default == 0].migrant_worker.dropna(), return_counts=True)
un = [str(int(i)) for i in un]
plt.bar(un, count, color = colors)
plt.xlabel("Migration Status")
plt.ylabel("Count")
plt.title("Are they non-defaulters an outsider?")
plt.subplot(2,2,3)
un, count = np.unique(data[data.credit_card_default == 1].migrant_worker.dropna(), return_counts=True)
un = [str(int(i)) for i in un]
plt.bar(un, count, color = colors)
plt.xlabel("Migration Status")
plt.ylabel("Count")
plt.title("Are they defaulters an outsider?")
plt.tight_layout()

There is a small variation in the migration status for Defaulters, where there are comparatively more Outsiders.

---

How much is their debt?

In [ ]:
plt.figure(figsize=(15,9))
plt.subplot(2,2,1)
plt.hist(data.yearly_debt_payments, color="lightblue")
plt.xlabel("Debt Amount Range")
plt.ylabel("Count")
plt.title("How much debt do they have?")
plt.subplot(2,2,2)
plt.hist(data[data.credit_card_default == 1].yearly_debt_payments, color="lightblue")
plt.xlabel("Debt Amount Range")
plt.ylabel("Count")
plt.title("How much debt do defaulters have?")
plt.subplot(2,2,3)
plt.hist(data[data.credit_card_default == 0].yearly_debt_payments, color="lightblue")
plt.xlabel("Debt Amount Range")
plt.ylabel("Count")
plt.title("How much debt do non-defaulters have?")
plt.tight_layout()

The debt status of both classes are similar with negligible variation

---

What is their credit limit?

In [ ]:
plt.figure(figsize=(10,9))
plt.subplot(2,2,1)
plt.ticklabel_format(useOffset=False)
plt.hist(data.credit_limit, color="skyblue")
plt.xlabel("Available Credit Limit Range")
plt.ylabel("Count")
plt.title("How much credit limit do they have?")

plt.subplot(2,2,2)
plt.ticklabel_format(useOffset=False)
plt.hist(data[data.credit_card_default == 1].credit_limit, color="skyblue")
plt.xlabel("Available Credit Limit Range")
plt.ylabel("Count")
plt.title("How much credit limit do defaulters have?")

plt.subplot(2,2,3)
plt.ticklabel_format(useOffset=False)
plt.hist(data[data.credit_card_default == 0].credit_limit, color="skyblue")
plt.xlabel("Available Credit Limit Range")
plt.ylabel("Count")
plt.title("How much credit limit do non-defaulters have?")
plt.tight_layout()

How much credit limit they used?

In [ ]:
plt.figure(figsize=(10,9))
plt.subplot(2,2,1)
plt.hist(data['credit_limit_used(%)'], color="darkgreen")
plt.xlabel("Used Credit Limit Range")
plt.ylabel("Count")
plt.title("How much credit limit have they used?")
plt.subplot(2,2,2)
plt.hist(data[data.credit_card_default == 1]['credit_limit_used(%)'], color="darkgreen")
plt.xlabel("Used Credit Limit Range")
plt.ylabel("Count")
plt.title("How much credit limit have defaulters used?")
plt.subplot(2,2,3)
plt.hist(data[data.credit_card_default == 0]['credit_limit_used(%)'], color="darkgreen")
plt.xlabel("Used Credit Limit Range")
plt.ylabel("Count")
plt.title("How much credit limit have non-defaulters used?")
plt.tight_layout()

What is their credit score?

In [ ]:
shuffle(colors)
plt.figure(figsize=(10,9))
plt.subplot(2,2,1)
plt.hist(data.credit_score, color=colors[0])
plt.xlabel("Credit Score Range")
plt.ylabel("Count")
plt.title("How much credit score do they have?")
plt.subplot(2,2,2)
plt.hist(data[data.credit_card_default == 1].credit_score, color=colors[0])
plt.xlabel("Credit Score Range")
plt.ylabel("Count")
plt.title("How much credit score do defaulters have?")
plt.subplot(2,2,3)
plt.hist(data[data.credit_card_default == 0].credit_score, color=colors[0])
plt.xlabel("Credit Score Range")
plt.ylabel("Count")
plt.title("How much credit score do non-defaulters have?")
plt.tight_layout()

Inference : The credit score of Defaulters range from 500 - 700 while for Non-Defaulters range from 650 - 950.


How much time did they default before?


In [ ]:
shuffle(colors)
plt.figure(figsize=(10,9))
plt.subplot(2,2,1)
un, count = np.unique(data.prev_defaults, return_counts=True)
un = [str(int(i)) for i in un]
plt.bar(un, count, color=colors)
plt.xlabel("Previous Default Count")
plt.ylabel("Count")
plt.title("How many times have they defaulted before?")
plt.subplot(2,2,2)
un, count = np.unique(data[data.credit_card_default == 1].prev_defaults, return_counts=True)
un = [str(int(i)) for i in un]
plt.bar(un, count, color=colors)
plt.xlabel("Previous Default Count")
plt.ylabel("Count")
plt.title("How many times have the defaulters defaulted before?")
plt.subplot(2,2,3)
un, count = np.unique(data[data.credit_card_default == 0].prev_defaults, return_counts=True)
un = [str(int(i)) for i in un]
plt.bar(un, count, color=colors)
plt.xlabel("Previous Default Count")
plt.ylabel("Count")
plt.title("How many times have the non-defaulters defaulted before?")
plt.tight_layout()

Inference : Majority of the Defaulters have defaulted Atleast previously.


Have they defaulted recently?

In [ ]:
shuffle(colors)
plt.figure(figsize=(10,9))
plt.subplot(2,2,1)
un, count = np.unique(data.default_in_last_6months, return_counts=True)
un = [str(int(i)) for i in un]
plt.bar(un, count, color=colors)
plt.xlabel("Recent Default Count")
plt.ylabel("Count")
plt.title("Have they defaulted recently?")
plt.subplot(2,2,2)
un, count = np.unique(data[data.credit_card_default == 1].default_in_last_6months, return_counts=True)
un = [str(int(i)) for i in un]
plt.bar(un, count, color=colors)
plt.xlabel("Recent Default Count")
plt.ylabel("Count")
plt.title("Have the defaulters defaulted recently?")
plt.subplot(2,2,3)
un, count = np.unique(data[data.credit_card_default == 0].default_in_last_6months, return_counts=True)
un = [str(int(i)) for i in un]
plt.bar(un, count, color=colors)
plt.xlabel("Recent Default Count")
plt.ylabel("Count")
plt.title("Have the non-defaulters defaulted recently?")
plt.tight_layout()

Inference : Majority of the defaulters have Defaulted in the last 6 months.

How much Profit / Loss occurs to them?

In [ ]:
plt.figure(figsize=(20,9))
plt.subplot(2,2,1)
shuffle(colors)
plt.xlabel("In hand profit range")
plt.ylabel("Count")
plt.title("In what range does loss occur to defaulters?")
plt.ticklabel_format(useOffset=False)
plt.hist([i for i in data[data.credit_card_default == 1].in_hand_balance if i < 0], color=colors[0])

plt.subplot(2,2,2)
plt.xlabel("In hand profit range")
plt.ylabel("Count")
plt.title("In what range does profit occur to defaulters?")
plt.ticklabel_format(useOffset=False)
plt.hist([i for i in data[data.credit_card_default == 1].in_hand_balance if i >= 0], color=colors[0])

plt.subplot(2,2,3)
plt.xlabel("In hand profit range")
plt.ylabel("Count")
plt.title("In what range does loss occur to non-defaulters?")
plt.ticklabel_format(useOffset=False)
plt.hist([i for i in data[data.credit_card_default == 0].in_hand_balance if i < 0], color=colors[0])

plt.subplot(2,2,4)
plt.xlabel("In hand profit range")
plt.ylabel("Count")
plt.title("In what range does profit occur to non-defaulters?")
plt.ticklabel_format(useOffset=False)
plt.hist([i for i in data[data.credit_card_default == 0].in_hand_balance if i >= 0], color=colors[0])

plt.tight_layout()

Inference : For Defaulters, the loss seems to be a higher number in comparison with Non-Defaulters, but counter intuitively, In hand profit appears Higher for Defaulters.


How many are saving?


In [ ]:
shuffle(colors)
plt.figure(figsize=(10,9))
plt.subplot(2,2,1)
un, count = np.unique(data.in_profit, return_counts=True)
plt.bar(un, count, color=colors)
plt.title("How many are able to save money?")
plt.xlabel("Profit status")
plt.ylabel("Count")
plt.subplot(2,2,2)
un, count = np.unique(data[data.credit_card_default == 1].in_profit, return_counts=True)
plt.bar(un, count, color=colors)
plt.title("How many defaulters are able to save money?")
plt.xlabel("Profit status")
plt.ylabel("Count")
plt.subplot(2,2,3)
un, count = np.unique(data[data.credit_card_default == 0].in_profit, return_counts=True)
plt.bar(un, count, color=colors)
plt.title("How many non-defaulters are able to save money?")
plt.xlabel("Profit status")
plt.ylabel("Count")
plt.tight_layout()

Inference : All of them are making a profit, with negligibly less Loss makers.

How much do they earn?


In [ ]:
shuffle(colors)
plt.figure(figsize=(15,9))
plt.subplot(2,2,1)
plt.hist(data.total_income_received, color=colors[0])
plt.xlabel("Income Range")
plt.ylabel("Count")
plt.title("How much do they earn?")

plt.subplot(2,2,2)
plt.hist(data[data.credit_card_default == 1].total_income_received, color=colors[0])
plt.xlabel("Income Range")
plt.ylabel("Count")
plt.title("How much do defaulters earn?")

plt.subplot(2,2,3)
plt.hist(data[data.credit_card_default == 0].total_income_received, color=colors[0])
plt.xlabel("Income Range")
plt.ylabel("Count")
plt.title("How much do non-defaulters earn?")
plt.tight_layout()

Inference : Defaulters are having a slightly higher income range in comparison with Non-Defaulters.

How many defaulted?

In [ ]:
shuffle(colors)
un, count = np.unique(data.credit_card_default, return_counts=True)
un = [str(int(i)) for i in un]
plt.bar(un, count, color = colors)
plt.xlabel("Default")
plt.ylabel("Count")
plt.title("How many were defaulters?")
plt.show()

Modelling:
Encoding Categorical Values
Oversampling using SMOTE
Modelling using Random Forest


Drop columns that are not required

In [ ]:
data.drop(["customer_id","name"],inplace=True,axis=1)

Label Encode Categorical Values

In [ ]:
enc = LabelEncoder()
data.gender = enc.fit_transform(data.gender)
data.owns_house = enc.fit_transform(data.owns_house)
data.owns_car = enc.fit_transform(data.owns_car)
data.occupation_type = enc.fit_transform(data.occupation_type)
data.migrant_worker = enc.fit_transform(data.migrant_worker)
data.in_profit = enc.fit_transform(data.in_profit)
data.credit_card_default = enc.fit_transform(data.credit_card_default)

Oversampling using SMOTE

In [ ]:
features, labels = SMOTE().fit_resample(data.drop(["credit_card_default"],axis=1), data.credit_card_default)

Modelling

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(features,
                                                    labels,
                                                    test_size = 0.2,
                                                    shuffle=True)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
rf = RandomForestClassifier(random_state=0,n_estimators=1000)
rf.fit(X_train, Y_train)

RandomForestClassifier(n_estimators=1000, random_state=0)

Metrics

In [ ]:
print("Train Accuracy  : {:.2f}%".format(accuracy_score(rf.predict(X_train),Y_train)*100))
print("Test Accuracy   : {:.2f}%".format(accuracy_score(rf.predict(X_test),Y_test)*100))
print("Precision       : {:.2f}%".format(precision_score(rf.predict(X_test),Y_test)*100))
print("Recall          : {:.2f}%".format(recall_score(rf.predict(X_test),Y_test)*100))

Train Accuracy  : 100.00%
Test Accuracy   : 98.74%
Precision       : 99.21%
Recall          : 98.33%

Confusion Matrix

In [ ]:
cm = confusion_matrix(Y_test,rf.predict(X_test))
disp = ConfusionMatrixDisplay(cm, display_labels=["1","0"])
disp.plot()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0.0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0.0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0.0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0.0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182324,125352.0,1.692781,-0.201306,-2.519907,1.461904,0.701510,-1.542165,1.432738,-0.730228,-0.371054,...,0.433586,0.993490,-0.425314,0.049406,0.824836,-0.217694,-0.091423,-0.041195,216.88,0.0
182325,125352.0,-0.507947,-0.546918,2.291673,-2.112876,-0.845382,-0.425270,-0.858402,0.256730,-0.103172,...,0.512459,1.377962,-0.181414,-0.003468,-0.405188,-0.195736,0.166438,0.161655,0.02,0.0
182326,125353.0,-1.841495,-1.506387,0.115844,0.057029,-0.202438,-0.100570,0.289007,0.592253,0.040401,...,0.715507,1.007318,0.636655,-0.572371,-0.692105,-0.300111,0.046361,-0.025163,327.28,0.0
182327,125353.0,-2.134618,0.341463,1.033744,-0.368409,0.547263,0.069410,0.439602,-0.292193,2.044366,...,-0.289225,0.605464,-0.323988,0.651900,1.004900,0.786506,-0.757787,-0.813204,12.85,0.0
